# Graph Analytics Serverless for any data source

<a target="_blank" href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/graph-analytics-serverless-standalone.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This Jupyter notebook is hosted [here](https://github.com/neo4j/graph-data-science-client/blob/main/examples/graph-analytics-serverless-standalone.ipynb) in the Neo4j Graph Data Science Client Github repository.

The notebook shows how to use the `graphdatascience` Python library to create, manage, and use a GDS Session.

We consider a graph of people and fruits, which we're using as a simple example to show how to load data from Pandas `DataFrame` to a GDS Session, run algorithms, and inspect the results. 
We will cover all management operations: creation, listing, and deletion.

If you are using AuraDB, follow [this example](../graph-analytics-serverless).
If you are using a self-managed Neo4j instance, follow [this example](../graph-analytics-serverless-self-managed).

## Prerequisites

This notebook requires having a Neo4j instance instance available and that the Graph Analytics Serverless [feature](https://neo4j.com/docs/aura/graph-analytics/#aura-gds-serverless) is enabled for your Neo4j Aura project. 

We also need to have the `graphdatascience` Python library installed, version `1.15` or later.

In [ ]:
%pip install "graphdatascience>=1.15"

## Aura API credentials

A GDS Session is managed via the Aura API.
In order to use the Aura API, we need to have [Aura API credentials](https://neo4j.com/docs/aura/platform/api/authentication/#_creating_credentials).

Using these credentials, we can create our `GdsSessions` object, which is the main entry point for managing GDS Sessions.

In [ ]:
import os

from graphdatascience.session import AuraAPICredentials, GdsSessions

client_id = os.environ["AURA_API_CLIENT_ID"]
client_secret = os.environ["AURA_API_CLIENT_SECRET"]

# If your account is a member of several projects, you must also specify the project ID to use
project_id = os.environ.get("AURA_API_PROJECT_ID", None)

sessions = GdsSessions(api_credentials=AuraAPICredentials(client_id, client_secret, project_id=project_id))

## Creating a new session

A new session is created by calling `sessions.get_or_create()`.
We also need to specify the session `memory` and `cloud_location`.

Please refer to the API reference [documentation](https://neo4j.com/docs/graph-data-science-client/current/api/sessions/gds_sessions/#graphdatascience.session.gds_sessions.GdsSessions.get_or_create) or the manual for a full list options.

Finally, we need to give our session a name.
We will call ours `people-and-fruits-standalone`.
It is possible to reconnect to an existing session by calling `get_or_create` with the same session name and configuration.

We will also set a time-to-live (TTL) for the session. 
This ensures that our session is automatically deleted after being unused for 30 minutes.
This is a good practice to avoid incurring costs should we forget to delete the session ourselves.

In [ ]:
from graphdatascience.session import AlgorithmCategory, CloudLocation, SessionMemory

# Explicitly define the size of the session
memory = SessionMemory.m_4GB

# Estimate the memory needed for the GDS session
memory = sessions.estimate(
    node_count=20,
    relationship_count=50,
    algorithm_categories=[AlgorithmCategory.CENTRALITY, AlgorithmCategory.NODE_EMBEDDING],
)

print(f"Estimated memory: {memory}")

# Specify your cloud location
cloud_location = CloudLocation("gcp", "europe-west1")

# You can find available cloud locations by calling
cloud_locations = sessions.available_cloud_locations()
print(f"Available locations: {cloud_locations}")

In [ ]:
from datetime import timedelta

# Create a GDS session!
gds = sessions.get_or_create(
    # we give it a representative name
    session_name="people-and-fruits-standalone",
    memory=memory,
    ttl=timedelta(minutes=30),
    cloud_location=cloud_location,
)

## Listing sessions

Now that we have created a session, let's list all our sessions to see what that looks like

In [ ]:
from pandas import DataFrame

gds_sessions = sessions.list()

# for better visualization
DataFrame(gds_sessions)

## Adding a dataset

We assume that the configured Neo4j database instance is empty.
We will add our dataset using standard Cypher.

In a more realistic scenario, this step is already done, and we would just connect to the existing database.

In [ ]:
import pandas as pd

people_df = pd.DataFrame(
    [
        {"nodeId": 0, "name": "Dan", "age": 18, "experience": 63, "hipster": 0},
        {"nodeId": 1, "name": "Annie", "age": 12, "experience": 5, "hipster": 0},
        {"nodeId": 2, "name": "Matt", "age": 22, "experience": 42, "hipster": 0},
        {"nodeId": 3, "name": "Jeff", "age": 51, "experience": 12, "hipster": 0},
        {"nodeId": 4, "name": "Brie", "age": 31, "experience": 6, "hipster": 0},
        {"nodeId": 5, "name": "Elsa", "age": 65, "experience": 23, "hipster": 0},
        {"nodeId": 6, "name": "Bobby", "age": 38, "experience": 4, "hipster": 1},
        {"nodeId": 7, "name": "John", "age": 4, "experience": 100, "hipster": 0},
    ]
)
people_df["labels"] = "Person"

fruits_df = pd.DataFrame(
    [
        {"nodeId": 8, "name": "Apple", "tropical": 0, "sourness": 0.3, "sweetness": 0.6},
        {"nodeId": 9, "name": "Banana", "tropical": 1, "sourness": 0.1, "sweetness": 0.9},
        {"nodeId": 10, "name": "Mango", "tropical": 1, "sourness": 0.3, "sweetness": 1.0},
        {"nodeId": 11, "name": "Plum", "tropical": 0, "sourness": 0.5, "sweetness": 0.8},
    ]
)
fruits_df["labels"] = "Fruit"

like_relationships = [(0, 8), (1, 9), (2, 10), (3, 10), (4, 9), (5, 11), (7, 11)]
likes_df = pd.DataFrame([{"sourceNodeId": src, "targetNodeId": trg} for (src, trg) in like_relationships])
likes_df["relationshipType"] = "LIKES"

knows_relationship = [(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 5), (7, 3)]
knows_df = pd.DataFrame([{"sourceNodeId": src, "targetNodeId": trg} for (src, trg) in knows_relationship])
knows_df["relationshipType"] = "KNOWS"

## Construct Graph from DataFrames

Now that we have imported a graph to our database, we create graphs directly from pandas `DataFrame` objects.
We do that by using the `gds.graph.construct()` method.

In [ ]:
# Dropping `name` column as GDS does not support string properties
nodes = [people_df.drop(columns="name"), fruits_df.drop(columns="name")]
relationships = [likes_df, knows_df]

G = gds.graph.construct("people-fruits", nodes, relationships)
str(G)

## Running Algorithms

We can now run algorithms on the constructed graph.
This is done using the standard GDS Python Client API.
There are many other tutorials covering some interesting things we can do at this step, so we will keep it rather brief here.

We will simply run PageRank and FastRP on the graph.


In [ ]:
print("Running PageRank ...")
pr_result = gds.pageRank.mutate(G, mutateProperty="pagerank")
print(f"Compute millis: {pr_result['computeMillis']}")
print(f"Node properties written: {pr_result['nodePropertiesWritten']}")
print(f"Centrality distribution: {pr_result['centralityDistribution']}")

print("Running FastRP ...")
frp_result = gds.fastRP.mutate(
    G,
    mutateProperty="fastRP",
    embeddingDimension=8,
    featureProperties=["pagerank"],
    propertyRatio=0.2,
    nodeSelfInfluence=0.2,
)
print(f"Compute millis: {frp_result['computeMillis']}")
# stream back the results
result = gds.graph.nodeProperties.stream(G, ["pagerank", "fastRP"], separate_property_columns=True)

result

To resolve each `nodeId` to name, we can merge it back with the source data frames.

In [ ]:
names = pd.concat([people_df, fruits_df])[["nodeId", "name"]]
result.merge(names, how="left")

## Deleting the session

Now that we have finished our analysis, we can delete the session.
The results that we produced were written back to our Neo4j database, and will not be lost.
If we computed additional things that we did not write back, those will be lost.

Deleting the session will release all resources associated with it, and stop incurring costs.

In [ ]:
# or gds.delete()
sessions.delete(session_name="people-and-fruits-standalone")

In [ ]:
# let's also make sure the deleted session is truly gone:
sessions.list()

## Conclusion

And we're done!
We have created a GDS Session, projected a graph, run some algorithms, inspect the results, and deleted the session.
This is a simple example, but it shows the main steps of using GDS Sessions.